In [110]:
# Define Node and Leaf classes
class Node:
    def __init__(self, left, value, right):
        self.left = left
        self.value = value
        self.right = right

    def __repr__(self):
        # Check if left and right are Node instances and extract their values
        left_value = self.left.value if isinstance(self.left, Node) else self.left
        right_value = self.right.value if isinstance(self.right, Node) else self.right

        #Return a simplified representation if left and right are not Leafs
        if not isinstance(self.left, Leaf) or not isinstance(self.right, Leaf):
          return f"Node(L={repr(left_value)}, V={repr(self.value)}, R={repr(right_value)})"
        
        return f"Node(L={repr(self.left)}, V={repr(self.value)}, R={repr(self.right)})"

class Leaf:
    def __repr__(self):
        return "Leaf()"

# Define traversal and operation functions (corrected inorder_traversal)
def inorder_traversal(root):
    def aux(node, acc):
        if isinstance(node, Leaf):
            return acc
        # Traverse left subtree
        acc = aux(node.left, acc)
        # Add the current node's value
        acc.append(node.value)
        # Traverse right subtree
        acc = aux(node.right, acc)
        return acc

    return aux(root, [])

def postorder_traversal(tree):
    def loop(node, cont, acc):
        if isinstance(node, Leaf) or node is None:  # Equivalent to Leaf
            return cont(acc)
        else:
            def cont_left(acc_left):
                def cont_right(acc_right):
                    return cont([node.value] + acc_right)
                return loop(node.right, cont_right, acc_left)
            return loop(node.left, cont_left, acc)
    
    return list(reversed(loop(tree, lambda x: x, [])))

def preorder_traversal(t):
    def aux(stack, acc):
        if not stack:
            return acc
        head, *tail = stack
        if isinstance(head, Leaf):
            return aux(tail, acc)
        else:  # head is BiNode
            return aux([head.left, head.right] + tail, [head.value] + acc)

    return list(reversed(aux([t], [])))
def height(tree):
    def aux(node, k):
        if isinstance(node, Leaf):
            return k(0)
        
        def after_left(left_height):
            def after_right(right_height):
                return k(1 + max(left_height, right_height))
            return aux(node.right, after_right)
        return aux(node.left, after_left)
    # Our top-level continuation is the identity function
    def identity(x):
        return x

    return aux(tree, identity)

def level_order_traversal(root):
    def aux(queue, acc):
        if not queue:
            return acc

        head = queue[0]
        tail = queue[1:]

        new_acc = acc + [head.value]

        new_queue = tail
        if not isinstance(head.left, Leaf):
            new_queue = new_queue + [head.left]
        if not isinstance(head.right, Leaf):
            new_queue = new_queue + [head.right]
        
        return aux(new_queue, new_acc)

    if isinstance(root, Leaf):
        return []
    return aux([root], [])

def insert(tree, n):
    def insert_aux(current, value, k):
        if isinstance(current, Leaf):
            return k(Node(Leaf(), value, Leaf()))
        elif isinstance(current, Node):
            if value < current.value:
                def left_continuation(new_left):
                    return k(Node(new_left, current.value, current.right))
                return insert_aux(current.left, value, left_continuation)
            elif value > current.value:
                def right_continuation(new_right):
                    return k(Node(current.left, current.value, new_right))
                return insert_aux(current.right, value, right_continuation)
            else:
                return k(current)

    def root_continuation(root):
        return root

    return insert_aux(tree, n, root_continuation)


In [111]:
# Define test trees
empty_tree = Leaf()

single_node_tree = Node(Leaf(), 10, Leaf())

left_leaning_tree = Node(
    Node(
        Node(Leaf(), 1, Leaf()),
        2,
        Leaf()
    ),
    3,
    Leaf()
)

right_leaning_tree = Node(
    Leaf(),
    1,
    Node(
        Leaf(),
        2,
        Node(
            Leaf(),
            3,
            Leaf()
        )
    )
)

balanced_tree = Node(
    Node(
        Node(Leaf(), 1, Leaf()),
        2,
        Node(Leaf(), 3, Leaf())
    ),
    4,
    Node(
        Leaf(),
        5,
        Node(Leaf(), 6, Leaf())
    )
)

unbalanced_tree = Node(
    Node(
        Leaf(),
        1,
        Node(
            Leaf(),
            2,
            Leaf()
        )
    ),
    3,
    Leaf()
)

bst_tree = Leaf()
values_to_insert = [5, 3, 7, 2, 4, 6, 8]
for value in values_to_insert:
    bst_tree = insert(bst_tree, value)

# Define test functions
def test_inorder_traversal():
    print("Testing Inorder Traversal...")

    # Test 1: Empty Tree
    assert inorder_traversal(empty_tree) == [], "Failed: Inorder traversal of empty tree"

    # Test 2: Single Node
    assert inorder_traversal(single_node_tree) == [10], "Failed: Inorder traversal of single node tree"

    # Test 3: Left-Leaning Tree
    assert inorder_traversal(left_leaning_tree) == [1, 2, 3], "Failed: Inorder traversal of left-leaning tree"

    # Test 4: Right-Leaning Tree
    assert inorder_traversal(right_leaning_tree) == [1, 2, 3], "Failed: Inorder traversal of right-leaning tree"

    # Test 5: Balanced Tree
    assert inorder_traversal(balanced_tree) == [1, 2, 3, 4, 5, 6], "Failed: Inorder traversal of balanced tree"

    # Test 6: Unbalanced Tree
    assert inorder_traversal(unbalanced_tree) == [1, 2, 3], "Failed: Inorder traversal of unbalanced tree"

    # Test 7: BST Tree
    assert inorder_traversal(bst_tree) == [2, 3, 4, 5, 6, 7, 8], "Failed: Inorder traversal of BST tree"

    print("All Inorder Traversal tests passed!\n")

def test_postorder_traversal():
    print("Testing Postorder Traversal...")

    # Test 1: Empty Tree
    assert postorder_traversal(empty_tree) == [], "Failed: Postorder traversal of empty tree"

    # Test 2: Single Node
    assert postorder_traversal(single_node_tree) == [10], "Failed: Postorder traversal of single node tree"

    # Test 3: Left-Leaning Tree
    assert postorder_traversal(left_leaning_tree) == [1, 2, 3], "Failed: Postorder traversal of left-leaning tree"

    # Test 4: Right-Leaning Tree
    assert postorder_traversal(right_leaning_tree) == [3, 2, 1], "Failed: Postorder traversal of right-leaning tree"

    # Test 5: Balanced Tree
    assert postorder_traversal(balanced_tree) == [1, 3, 2, 6, 5, 4], "Failed: Postorder traversal of balanced tree"

    # Test 6: Unbalanced Tree
    assert postorder_traversal(unbalanced_tree) == [2, 1, 3], "Failed: Postorder traversal of unbalanced tree"

    # Test 7: BST Tree
    assert postorder_traversal(bst_tree) == [2, 4, 3, 6, 8, 7, 5], "Failed: Postorder traversal of BST tree"

    print("All Postorder Traversal tests passed!\n")

def test_preorder_traversal():
    print("Testing Preorder Traversal...")

    # Test 1: Empty Tree
    assert preorder_traversal(empty_tree) == [], "Failed: Preorder traversal of empty tree"

    # Test 2: Single Node
    assert preorder_traversal(single_node_tree) == [10], "Failed: Preorder traversal of single node tree"

    # Test 3: Left-Leaning Tree
    assert preorder_traversal(left_leaning_tree) == [3, 2, 1], "Failed: Preorder traversal of left-leaning tree"

    # Test 4: Right-Leaning Tree
    assert preorder_traversal(right_leaning_tree) == [1, 2, 3], "Failed: Preorder traversal of right-leaning tree"

    # Test 5: Balanced Tree
    assert preorder_traversal(balanced_tree) == [4, 2, 1, 3, 5, 6], "Failed: Preorder traversal of balanced tree"

    # Test 6: Unbalanced Tree
    assert preorder_traversal(unbalanced_tree) == [3, 1, 2], "Failed: Preorder traversal of unbalanced tree"

    # Test 7: BST Tree
    assert preorder_traversal(bst_tree) == [5, 3, 2, 4, 7, 6, 8], "Failed: Preorder traversal of BST tree"

    print("All Preorder Traversal tests passed!\n")

def test_height():
    print("Testing Height Function...")

    # Test 1: Empty Tree
    assert height(empty_tree) == 0, "Failed: Height of empty tree"

    # Test 2: Single Node
    assert height(single_node_tree) == 1, "Failed: Height of single node tree"

    # Test 3: Left-Leaning Tree
    assert height(left_leaning_tree) == 3, "Failed: Height of left-leaning tree"

    # Test 4: Right-Leaning Tree
    assert height(right_leaning_tree) == 3, "Failed: Height of right-leaning tree"

    # Test 5: Balanced Tree
    assert height(balanced_tree) == 3, "Failed: Height of balanced tree"

    # Test 6: Unbalanced Tree
    assert height(unbalanced_tree) == 3, "Failed: Height of unbalanced tree"

    # Test 7: BST Tree
    assert height(bst_tree) == 3, "Failed: Height of BST tree"

    print("All Height Function tests passed!\n")

def test_level_order_traversal():
    print("Testing Level Order Traversal...")

    # Test 1: Empty Tree
    assert level_order_traversal(empty_tree) == [], "Failed: Level order traversal of empty tree"

    # Test 2: Single Node
    assert level_order_traversal(single_node_tree) == [10], "Failed: Level order traversal of single node tree"

    # Test 3: Left-Leaning Tree
    assert level_order_traversal(left_leaning_tree) == [3, 2, 1], "Failed: Level order traversal of left-leaning tree"

    # Test 4: Right-Leaning Tree
    assert level_order_traversal(right_leaning_tree) == [1, 2, 3], "Failed: Level order traversal of right-leaning tree"

    # Test 5: Balanced Tree
    assert level_order_traversal(balanced_tree) == [4, 2, 5, 1, 3, 6], "Failed: Level order traversal of balanced tree"

    # Test 6: Unbalanced Tree
    assert level_order_traversal(unbalanced_tree) == [3, 1, 2], "Failed: Level order traversal of unbalanced tree"

    # Test 7: BST Tree
    assert level_order_traversal(bst_tree) == [5, 3, 7, 2, 4, 6, 8], "Failed: Level order traversal of BST tree"

    print("All Level Order Traversal tests passed!\n")

def test_insert_duplicates():
    print("Testing Insertions with Duplicates...")
    tree = Leaf()
    values = [5, 3, 7, 3, 5, 8]
    for v in values:
        tree = insert(tree, v)
    
    # Assuming duplicates are ignored, the BST should have unique values
    expected_inorder = [3, 5, 7, 8]
    result_inorder = inorder_traversal(tree)
    assert result_inorder == expected_inorder, f"Failed: Inorder after duplicates insertion {result_inorder}"
    
    # Check level order to ensure duplicates are not inserted
    expected_level_order = [5, 3, 7, 8]
    result_level_order = level_order_traversal(tree)
    assert result_level_order == [5, 3, 7, 8], f"Failed: Level order after duplicates insertion {result_level_order}"
    
    print("Insertions with Duplicates test passed!\n")

def test_large_tree():
    print("Testing Large Tree...")

    # Create a large BST with values from 1 to 20 (using 20 instead of 100 for practicality)
    large_tree = Leaf()
    for i in range(1, 21):
        large_tree = insert(large_tree, i)
    
    # Test inorder traversal
    expected_inorder = list(range(1, 21))
    result_inorder = inorder_traversal(large_tree)
    assert result_inorder == expected_inorder, "Failed: Inorder traversal of large tree"

    # Test height
    # Since we inserted in ascending order, the tree is right-leaning, so height should be 20
    expected_height = 20
    actual_height = height(large_tree)
    assert actual_height == expected_height, f"Failed: Height of large tree. Expected {expected_height}, got {actual_height}"

    print("Large Tree tests passed!\n")


In [113]:
def run_all_tests():
    test_inorder_traversal()
    test_postorder_traversal()
    test_preorder_traversal()
    test_height()
    test_level_order_traversal()
    test_insert_duplicates()
    # Note: Uncomment the following line to run the large tree test
    # Be cautious as it may raise a RecursionError
    # test_large_tree()
    print("All tests completed successfully!")
run_all_tests()

Testing Inorder Traversal...
All Inorder Traversal tests passed!



Testing Postorder Traversal...
All Postorder Traversal tests passed!

Testing Preorder Traversal...
All Preorder Traversal tests passed!

Testing Height Function...
All Height Function tests passed!

Testing Level Order Traversal...
All Level Order Traversal tests passed!

Testing Insertions with Duplicates...
Insertions with Duplicates test passed!

All tests completed successfully!
